In [1]:
import torch
import torch.nn as nn

from LineByLineNLPTextDataset import LineByLineNLPTextDataset
from LineByLineClassificationTextDataset import LineByLineClassificationTextDataset
from nlp.builder import FORCE_REDOWNLOAD

from transformers import BertForMaskedLM, BertTokenizerFast

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
PyTorch version 1.6.0 available.
Apache Beam available.


In [2]:
OUTPUT_DIR = f"../../weights/es-lstm/"
TRAIN_PATH = f"../../data/minicorpus1/train_1.txt"
TEST_PATH = f"../../data/minicorpus1/test_1.txt"
VAL_PATH = f"../../data/minicorpus1/val_1.txt"

In [3]:
#filter()
data_path = "../../data/spanish-corpora"

In [4]:
import os
data_path = "../../data/spanish-corpora"
TRAIN_PATHS = [os.path.join(data_path, filename) for filename in filter(lambda x: x.endswith((".txt")), os.listdir(data_path))]

In [5]:
TRAIN_PATHS

['../../data/spanish-corpora/preprocessed_JRC_lower.txt',
 '../../data/spanish-corpora/preprocessed_OpenSubtitles2018_lower.txt',
 '../../data/spanish-corpora/preprocessed_allwikis_lower.txt',
 '../../data/spanish-corpora/preprocessed_EMEA_lower.txt',
 '../../data/spanish-corpora/preprocessed_GlobalVoices_lower.txt',
 '../../data/spanish-corpora/preprocessed_DGT_lower.txt',
 '../../data/spanish-corpora/preprocessed_ParaCrawl_lower.txt',
 '../../data/spanish-corpora/preprocessed_NewsCommentary11_lower.txt',
 '../../data/spanish-corpora/preprocessed_EUBookShop_lower.txt',
 '../../data/spanish-corpora/preprocessed_DOGC_lower.txt',
 '../../data/spanish-corpora/preprocessed_Europarl_lower.txt',
 '../../data/spanish-corpora/preprocessed_multiUN_lower.txt',
 '../../data/spanish-corpora/preprocessed_TED_lower.txt',
 '../../data/spanish-corpora/preprocessed_UN_lower.txt',
 '../../data/spanish-corpora/preprocessed_ECB_lower.txt']

In [6]:
!ls ../../data/minicorpus1

test_1.txt  train_1.txt  val_1.txt


In [7]:
block_size = 128

In [7]:
tokenizer = BertTokenizerFast.from_pretrained("../../weights/beto/", do_lower_case=False)

In [9]:
dataset = LineByLineNLPTextDataset(tokenizer, TRAIN_PATHS, block_size, overwrite_cache=False)

Checking /home/camilojd/.cache/huggingface/datasets/3e34209a2741375a1db1ff03bf1abba1a9bd0e6016912d3ead0114b9d1ca2685.964a0948391d87479f79716750233e9a593a71bb70d468a93367cd00247e6451.py for additional imports.
Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py at /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text
Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py at /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28
Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py to /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28/text.py
Couldn't find dataset infos file at https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/dataset_infos.json
Found metadata file for dataset

Dataset not on Hf google storage. Downloading and preparing it from source
Unable to verify checksums.
Generating split train


Done writing 300904000 examples in 42477164603 bytes /home/camilojd/.cache/huggingface/datasets/text/dataset-1343f33c3d71378a/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28.incomplete/text-train.arrow.
Unable to verify splits sizes.
Constructing Dataset for split train, from /home/camilojd/.cache/huggingface/datasets/text/dataset-1343f33c3d71378a/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28


Dataset text downloaded and prepared to /home/camilojd/.cache/huggingface/datasets/text/dataset-1343f33c3d71378a/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28. Subsequent calls will reuse this data.


Unable to verify checksums.
Caching processed dataset at /home/camilojd/.cache/huggingface/datasets/text/dataset-1343f33c3d71378a/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28/cache-e84d071e5f4f445f.arrow


OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [4]:
from transformers.modeling_utils import PreTrainedModel
from transformers.configuration_utils import PretrainedConfig
from torch.autograd import Variable

class GenerativeLSTM(PreTrainedModel):
    def __init__(self, config):
        super(GenerativeLSTM, self).__init__(config)
        
        # Params
        self.input_size = 128
        self.hidden_size = 256
        self.num_layers = 3 
        self.output_size = 31002
        
        # Layers
        self.embedding = nn.Embedding(
            num_embeddings=self.output_size,
            embedding_dim=self.input_size
        )
        
        self.lstm = nn.LSTM(
            input_size=self.input_size, 
            hidden_size=self.hidden_size, 
            num_layers=self.num_layers, 
            #bidirectional=True, 
            batch_first=True,
            dropout=0.2
        )
        
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
        self.prev_state = None
        #self.tokenizer = BertTokenizerFast.from_pretrained("../../weights/beto/", do_lower_case=False)
        
    def forward(self, input_ids, labels=None):
        batch_size = len(input_ids)
        
        embed = self.embedding(input_ids)
        lstm_output, actual_state = self.lstm(embed, self.prev_state)
        self.prev_state = (actual_state[0].detach(), actual_state[1].detach())
        lm_logits = self.fc(lstm_output)
        
        outputs = (lm_logits,) + (lstm_output,)
        
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Reshape the tokens
            loss_fct = nn.CrossEntropyLoss()
            #loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss = loss_fct(
                shift_logits.view(shift_logits.size(0), shift_logits.size(-1), shift_logits.size(-2)), 
                shift_labels
            )
            outputs = (loss,) + outputs
            
        return outputs

In [ ]:
config = PretrainedConfig()
model = GenerativeLSTM(config)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    warmup_steps=10_000,
    max_steps=90_000,
    learning_rate=0.0001,
    fp16=True
)

"""
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=64,
    save_steps=1_00,
    save_total_limit=2,
    max_steps=1_000,
    learning_rate=0.0001,
    #fp16=True
)"""

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
%%time
trainer.train()

# Load Model

In [2]:
!ls ../../weights/es-lstm/checkpoint-60000

config.json  optimizer.pt  pytorch_model.bin  scheduler.pt  training_args.bin


In [5]:
config = PretrainedConfig()
model_pred = GenerativeLSTM(config)
model_pred.load_state_dict(torch.load("../../weights/es-lstm/checkpoint-60000/pytorch_model.bin"))
model_pred.eval()

GenerativeLSTM(
  (embedding): Embedding(31002, 128)
  (lstm): LSTM(128, 256, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=31002, bias=True)
)

In [8]:
sentence = tokenizer.encode("los signos más probables de", return_tensors="pt", add_special_tokens=True)
model_pred(sentence)[0].shape

torch.Size([1, 8, 31002])

In [19]:
sentence = tokenizer.encode("Buenos dias a todos, desean salir a", return_tensors="pt", add_special_tokens=True)
predictions = model_pred(sentence)[0]
idxs = torch.argsort(predictions[0, -1], descending=True)
predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
predicted_token

['##mart', 'Vigo', 'Escuelas', 'proporcionados', 'Front']

In [ ]:
sentence

In [ ]:
predictions

In [ ]:
[tokenizer.decode(sen) for sen in sentence]

In [ ]:
idxs = torch.argsort(predictions[0, -1], descending=True)
predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
predicted_token